In [0]:
%run ./00_config

In [0]:
# Using %run above to load shared configuration variables (account name, API key and etc)

In [0]:
# -------------------- League ID Resolver --------------------

def get_league_id(sport_name, default_id=None):
    try:
        return sports_ids[sport_name.lower()]
    except KeyError:
        print(f"[ERROR] Sport '{sport_name}' not found in local mapping.")
        print("Please refer to https://www.thesportsdb.com/documentation for valid sport names.")
        return default_id

In [0]:
# -------------------- API Call + Save --------------------

def fetch_and_save_season_events(sport_id, season_year):
    url = f"https://www.thesportsdb.com/api/v1/json/{API_KEY}/eventsseason.php?id={sport_id}&s={season_year}"

    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json().get("events", [])
        if not data:
            print("No data returned for the specified year or sport.")
            return

        file_path = f"{bronze_adls}/{INGESTION_DATE}_{sport_id}_season_data.json"
        json_data = json.dumps(data, indent=4)
        dbutils.fs.put(file_path, json_data, overwrite=True)
        print(f"Data successfully saved to {file_path}")
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from API: {e}")

In [0]:
# -------------------- Execute ETL --------------------

sport_id = get_league_id(sport_name)
fetch_and_save_season_events(sport_id, season_year)